In [1]:
%%capture
# will take approx 5 min
import os
if not os.path.exists('open-image-bus-trucks'):
  %pip install torch_snippets utlralytics
  !wget --quiet https://www.dropbox.com/s/agmzwk95v96ihic/open-images-bus-trucks.tar.xz
  !tar -xf open-images-bus-trucks.tar.xz
  !rm open-images-bus-trucks.tar.xzd

In [2]:
!pip install torch_snippets

In [3]:
from torch_snippets import *
def prepare_yolo_data(image_folder, labels_folder, train_txt, val_txt, output_folder):
  def prepare_yolo_split(split, image_folder, labels_folder, txt, output_folder):
    items = readlines(txt)
    for item in items:
      item = stem(item)
      im_to = output_folder/'images'/split/f'{item}.jpg'
      ann_to = output_folder/'labels'/split/f'{item}.txt'
      makedir(parent(im_to)), makedir(parent(ann_to))
      im = image_folder/f'{item}.jpg'
      ann = labels_folder/f'{item}.txt'
      im.cp(im_to)
      ann.cp(ann_to)

  prepare_yolo_split('train', image_folder, labels_folder, train_txt, output_folder)
  prepare_yolo_split('val', image_folder, labels_folder, val_txt, output_folder)


data_version = 'all'
yolo_path = P(f'./yolo-open-images-bus-trucks-{data_version}')

try: yolo_path.rmtree(force=True)
except: ...

prepare_yolo_data(
  P('open-images-bus-trucks/images'),
  P(f'open-images-bus-trucks/yolo_labels/{data_version}/labels/'),
  P(f'open-images-bus-trucks/yolo_labels/{data_version}/train.txt/'),
  P(f'open-images-bus-trucks/yolo_labels/{data_version}/val.txt/'),
  P(yolo_path)
)

[10/24/24 11:07:16] INFO     loaded 12439 lines                                                                          ]8;id=68463;file://<ipython-input-3-24dcbd80ba49>:4\<ipython-input-3-24dcbd80ba49>]8;;\:]8;id=884805;file://<ipython-input-3-24dcbd80ba49>:4#prepare_yolo_split:4\prepare_yolo_split:4]8;;\

[10/24/24 11:07:30] INFO     loaded 5104 lines                                                                           ]8;id=741789;file://<ipython-input-3-24dcbd80ba49>:4\<ipython-input-3-24dcbd80ba49>]8;;\:]8;id=393701;file://<ipython-input-3-24dcbd80ba49>:4#prepare_yolo_split:4\prepare_yolo_split:4]8;;\

In [2]:
%%writefile bus_truck_all.yaml

path: /content/yolo-open-images-bus-trucks-all/ # dataset root dir
train: images/train  # train images (relative to 'path') 4 images
val: images/val # val images (relative to 'path') 4 images

# classes
names:
  0: bus
  1: truck

Overwriting bus_truck_all.yaml


In [5]:
!pip install ultralytics

In [ ]:
from ultralytics import YOLO

# load model
model = YOLO("yolov8m.pt")  # load a pretrained model (recommended for training)
_ = model.train(data=f"bus_truck_{data_version}.yaml", epochs=3)  # train the model

Ultralytics 8.3.21 🚀 Python-3.10.12 torch-2.5.0+cu121 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=bus_truck_all.yaml, epochs=3, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: ebilpaul06 (ebilpaul06-uganda-institute-of-information-comm-uict). Use `wandb login --relogin` to force relogin


Freezing layer 'model.22.dfl.conv.weight'


train: Scanning /content/yolo-open-images-bus-trucks-all/labels/train... 12439 images, 0 backgrounds, 0 corrupt: 100%|██████████| 12439/12439 [00:08<00:00, 1512.11it/s]


train: New cache created: /content/yolo-open-images-bus-trucks-all/labels/train.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.19 (you have 1.4.15). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
val: Scanning /content/yolo-open-images-bus-trucks-all/labels/val... 5104 images, 0 backgrounds, 0 corrupt: 100%|██████████| 5104/5104 [00:03<00:00, 1620.34it/s]


val: New cache created: /content/yolo-open-images-bus-trucks-all/labels/val.cache
Plotting labels to runs/detect/train2/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.0005), 83 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs/detect/train2
Starting training for 3 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


  0%|          | 0/778 [00:00<?, ?it/s]

In [1]:
# results = model("https://ultralytics.com/images/bus.jpg")  # predict on an image
val_images = Glob("/content/yolo-open-images-bus-trucks-mini/images/val")

for _ in range(10):
  results = model(choose(val_images))
  boxes = results[0].boxes.xyxy.cpu().numpy().astype(int)
  classes = [results[0].names[cls] for cls in results[0].boxes.cls.cpu().numpy()]
  show(results[0].org_img, bbs=boxes, texts=classes, sz=5, text_sz=10)



NameError: name 'Glob' is not defined